In [53]:
import pandas as pd

data_path = '/Users/mustafacavusoglu/workspace/mlops-course/data'

data = pd.read_parquet(f'{data_path}/yellow_tripdata_2023-01.parquet')
len(data.columns)

19

In [54]:
data['duration'] = data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime']
data['duration'] = data['duration'].dt.total_seconds() / 60

In [55]:
# Q2
#std dev of duration
std_dev = data['duration'].std()
print(f'Standard deviation of duration: {std_dev}')

Standard deviation of duration: 42.59435124195458


In [56]:
# Q3
# filter out smalller than 1 minute and larger than 60 minutes
data_dropped = data[(data['duration'] >= 1) & (data['duration'] <= 60)]
data_dropped.shape[0] / data.shape[0] * 100

98.1220282212598

In [57]:
# Q4
from sklearn.feature_extraction import DictVectorizer

# Convert the DataFrame to a list of dictionaries
data_dicts = data_dropped[['PULocationID', 'DOLocationID']].to_dict(orient='records')
# Convert the IDs to strings
for record in data_dicts:
    record['PULocationID'] = str(record['PULocationID'])
    record['DOLocationID'] = str(record['DOLocationID'])
# Initialize the DictVectorizer
dv = DictVectorizer()
# Fit the DictVectorizer and transform the data
X = dv.fit_transform(data_dicts)
# Convert the sparse matrix to a dense format
X_dense = X.toarray()
# Convert the dense matrix to a DataFrame
X_df = pd.DataFrame(X_dense, columns=dv.get_feature_names_out())
# Display the first few rows of the DataFrame
X_df.shape

(3009173, 515)

In [59]:
# Q5

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X, data_dropped['duration'])
y_pred = model.predict(X)
rmse = mean_squared_error(data_dropped['duration'], y_pred)
print(f'RMSE on train: {rmse ** 0.5}')


RMSE on train: 7.649262183753913


In [60]:
# Q6

# validation set
data_val = pd.read_parquet(f'{data_path}/yellow_tripdata_2023-02.parquet')
data_val['duration'] = data_val['tpep_dropoff_datetime'] - data_val['tpep_pickup_datetime']
data_val['duration'] = data_val['duration'].dt.total_seconds() / 60
data_val_dropped = data_val[(data_val['duration'] >= 1) & (data_val['duration'] <= 60)]
data_val_dicts = data_val_dropped[['PULocationID', 'DOLocationID']].to_dict(orient='records')
for record in data_val_dicts:
    record['PULocationID'] = str(record['PULocationID'])
    record['DOLocationID'] = str(record['DOLocationID'])
X_val = dv.transform(data_val_dicts)
y_val_pred = model.predict(X_val)
rmse_val = mean_squared_error(data_val_dropped['duration'], y_val_pred)
print(f'RMSE on validation: {rmse_val ** 0.5}')

RMSE on validation: 7.811812092681157
